In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

### 1. 准备数据，有两个数据格式有问题，因此先使用excel方式读取，再保存为csv文件

In [16]:
cj_data = pd.read_csv("data1/bks_cjxx_out.csv")
cj_data.to_csv("bks_cjxx_out.csv", index = None)

In [18]:
xjjbsj_data = pd.read_excel("data1/bks_xjjbsjxx_out.csv")
kcsj_data = pd.read_csv("data1/bks_kcsjxx_out.csv")

kcsj_data.to_csv("data1/bks_kcsjxx_out.csv", index = None)
xjjbsj_data.to_csv("data1/bks_xjjbsjxx_out.csv", index = None)

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'xh,yxsh,'

In [19]:
data1 = pd.read_csv("data1/cjxx_out1.csv",encoding = "GBK") 
data2 = pd.read_csv("data1/cjxx_out2.csv",encoding = "GBK",names = data1.columns) 
data12 = pd.concat([data1, data2], ignore_index = True)


xjjbsj_data = pd.read_excel("data1/bks_xjjbsjxx_out.xlsx")
kcsj_data = pd.read_csv("data1/bks_kcsjxx_out.csv")
kcsj_data.rename(columns={'zxs':'zxxs'}, inplace = True)

data12.to_csv("bks_cjxx_out.csv", index = None)
kcsj_data.to_csv("data1/bks_kcsjxx_out.csv", index = None)
xjjbsj_data.to_csv("data1/bks_xjjbsjxx_out.csv", index = None)

### 2. 读取数据，分别为  成绩， 课程数据， 排课数据， 学籍基本数据， 学生基本数据， 一卡通消费日志，并整合放入file_list中

In [2]:
cj_data = pd.read_csv("data1/bks_cjxx_out.csv")   #成绩
kcsj_data = pd.read_csv("data1/bks_kcsjxx_out.csv")              #课程数据
pksj_data = pd.read_csv("data1/bks_pksjxx_out.csv")              #排课数据
xjjbsj_data = pd.read_csv("data1/bks_xjjbsjxx_out.csv")   #学籍基本数据
xsjbsj_data = pd.read_csv("data1/bks_xsjbsjxx_out.csv")          #学生基本数据
jyrz_data = pd.read_table('data1/ykt_jyrz_lishi.txt',sep = ';')  #一卡通消费日志    交易金额和交易余额需要除以100


file_list = [cj_data, kcsj_data, pksj_data, xjjbsj_data, xsjbsj_data, jyrz_data]
file_list_name = ['成绩表', '数据课程信息','排课数据信息', '学籍基本数据信息', '学生基本数据信息', '一卡通消费日志']

### 3. 构建maps表和get_value()函数  之后所有查看标签含义都可以通过调用get_value(), e.g. get_value("JYYE")

In [4]:
#构建maps表
maps = {}
maps_list = []
data = pd.read_excel("data1/数据结构说明.xlsx", sheet_name = None)
for sheet in data:
    maps_list.append(sheet)
maps_list.remove(maps_list[0])
for i in range(6):
    for key, value in zip(data[maps_list[i]].iloc[:,0], data[maps_list[i]].iloc[:,2]):
        key = key.replace(' ','')
        maps[key] = value

#定义获取value函数
def get_value(key):
    if key.lower() not in maps:
        print('get_value收到了一个不存在的key值', key)
        return None
    return maps[key.lower()]

### 4. 为每个文件修改标签名，方便查看数据

In [5]:
for file_index in range(len(file_list)):
    for i in range(len(file_list[file_index].columns)):
        if get_value(file_list[file_index].columns[i]) == None:
            print(file_index, '.', file_list[file_index].columns[i], ' is not found')
            continue;
        file_list[file_index].rename(columns={file_list[file_index].columns[i]:get_value(file_list[file_index].columns[i])}, inplace = True)

get_value收到了一个不存在的key值 Unnamed: 0
0 . Unnamed: 0  is not found
get_value收到了一个不存在的key值 rkjsgh
0 . rkjsgh  is not found
get_value收到了一个不存在的key值 Unnamed: 0
1 . Unnamed: 0  is not found
get_value收到了一个不存在的key值 zxs
2 . zxs  is not found


### 删除全空属性     总学时有bug

In [6]:
for file_index in range(len(file_list)):
    for column in file_list[file_index].columns:
        if column == '总学时':
            continue;
        if file_list[file_index][column].count() == 0:
            print(file_index, '.', column, 'is nan')
            file_list[file_index].drop(column, axis = 1, inplace=True)

0 . 考试性质码 is nan
0 . 考试形式码 is nan
0 . 课程等级成绩码 is nan
0 . rkjsgh is nan
0 . 是否重修 is nan
0 . 教学班号 is nan
0 . 重修补考标志 is nan
1 . 周学时 is nan
1 . 课程类别代码 is nan
2 . 上课时间 is nan
2 . 课容量 is nan
2 . 选课校区号 is nan
2 . 选课人数限定 is nan
2 . 选课年级 is nan
2 . 教室类型码 is nan
2 . 课程性质码 is nan
2 . 上课班级号 is nan
2 . 教学班名称 is nan
2 . 教学班号 is nan
2 . 教学资源 is nan
2 . 开课说明 is nan
2 . 排课要求 is nan
2 . 计划课程编号 is nan
2 . 教学特征 is nan
2 . 是否开课 is nan
2 . zxs is nan
3 . 培养方式码 is nan
3 . 连读方式码 is nan
3 . 学生层次码 is nan
4 . 籍贯 is nan


### 打印可用属性

In [7]:
for file_index in range(len(file_list)):
    print(file_list_name[file_index])
    for column in file_list[file_index].columns:
        print( '▪', column)
    print('\n\n\n')

成绩表
▪ Unnamed: 0
▪ 学号
▪ 学年(度)
▪ 学期码
▪ 考试日期
▪ 课程号
▪ 平时成绩
▪ 分数类考试成绩
▪ 等级类考试成绩
▪ 课程成绩
▪ 成绩编号
▪ 学分
▪ 绩点
▪ 课程属性代码(001，必修；002，选修；003，任选；004，辅修；005，限选)
▪ 修读方式代码(01，正常；04，重修；06，复修；08，补考 )
▪ 课序号（新增）
▪ 成绩范围.ln,历年成绩库；lr，当前录入成绩库
▪ 考试类别码(01，集中周；02，非集中周；03，国家考试)
▪ 等级成绩
▪ 成绩录入方式代码（1027新增）(001	录入百分成绩,002 录入等级成绩,003 录入明细成绩)
▪ 替代课程号（1027新增）




数据课程信息
▪ Unnamed: 0
▪ 课程号
▪ 课程名称
▪ 课程英文名称
▪ 学分
▪ 总学时
▪ 教材
▪ 理论学时
▪ 实验学时
▪ 课程开设单位号
▪ 实践学时
▪ 参考书目
▪ 课程简介
▪ 编号
▪ 开课校区




排课数据信息
▪ 课程开设单位号
▪ 开课学年度
▪ 开课学期码
▪ 教学地点
▪ 修读人数
▪ 教室所在校区号
▪ 起始周
▪ 终止周
▪ 排课编号
▪ 数据同步时间
▪ 总学时
▪ 课程号
▪ 上课星期（标识星期几上课）
▪ 上课节次
▪ 持续节次
▪ 上课周次
▪ 周次说明
▪ 校区号
▪ 教学计划编号
▪ 教学楼号
▪ 教室号
▪ 课序号（新增）




学籍基本数据信息
▪ 学号
▪ 院系所号
▪ 专业码
▪ 学科门类码
▪ 学生类别码
▪ 所在年级
▪ 培养层次码（必填项：3  本科
4  专科
9  其他）
▪ 学生当前状态码
▪ 所在班号
▪ 入学年月
▪ 是否有学籍(新增)
▪ 是否有国家学籍(新增)
▪ 校区号
▪ 学制（新增）




学生基本数据信息
▪ 学号
▪ 姓名
▪ 性别码
▪ 出生日期
▪ 出生地码
▪ 民族码
▪ 国籍/地区码
▪ 港澳台侨外码
▪ 政治面貌码
▪ 健康状况码
▪ 信仰宗教码
▪ 是否留学生
▪ 是否独生子女




一卡通消费日志
▪ 学号
▪ 交易类型
▪ 交易金额
▪ 交易日期
▪ 交易时间
▪ 交易地点
▪ 商户代码
▪ 商户名称
▪  pos编号
▪  累计刷卡次数
▪ 交易余额






In [14]:
#file_list = [cj_data, kcsj_data, pksj_data, xjjbsj_data, xsjbsj_data, jyrz_data]
#file_list_name = ['成绩表', '数据课程信息','排课数据信息', '学籍基本数据信息', '学生基本数据信息', '一卡通消费日志']
jyrz_data['交易地点'].unique()

array(['西山浴室', '圈存缴网费', '西山热水器', '校医院', '体育馆', '中心食堂一层', '五食堂一层',
       '中心食堂二层', '大学生超市', '中心食堂三层', '京鹤直饮水', '开发区校区食堂', '开发区浴室', '五食堂二层',
       '电子缴电费', '沁园餐厅', '开发区开水机', '卢工洗衣机', '班车', '五食堂三层', '郭顺发洗衣机',
       '中心食堂清真', '教育书店咖啡厅', '机房管理商户', '开发区网络计费', '博留一层', '七食堂1楼', '梁工洗衣机',
       '七食堂2楼', '开发区智能控电', '教务注册大厅收费', '汇文系统', '校友工作处', '软件学院体育馆', '建艺学院',
       '九食堂清真餐厅', '十食堂超市', '盘锦校区B12食堂', '博留二层', '图书馆机房管理系统', '博留咖啡厅',
       '北山AB区热水器', '北山B区浴室', '盘锦校区B13食堂'], dtype=object)